# Camada Bronze - Arquitetura Medalhão

**Objetivo:** Ingestão de dados brutos da camada Landing para Bronze em formato Delta Lake.

**Operações:**
- Leitura de 8 arquivos CSV do volume Landing
- Adição de coluna `ingestion_timestamp` para rastreabilidade
- Gravação em tabelas Delta gerenciadas
- Validações de qualidade e integridade dos dados

**Padrão de Nomenclatura:**
- `ft_` = Tabelas Fato (fact tables)
- `dm_` = Tabelas Dimensão (dimension tables)

## Setup Inicial

Criação da estrutura do Unity Catalog necessária para armazenar os dados.
Este passo garante que o catálogo, schemas e volumes existam antes da ingestão.

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS atendimento_catalog;

In [0]:
%sql
USE CATALOG atendimento_catalog;
    
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;

In [0]:
%sql
USE SCHEMA default;
CREATE VOLUME IF NOT EXISTS landing;

## Configuração Inicial

Definição de variáveis e configuração do ambiente Spark.
Importação das bibliotecas necessárias e definição dos paths de origem e destino dos dados.

In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col, lit
import pyspark.sql.functions as F

In [0]:
catalog_name = "atendimento_catalog"
bronze_db_name = "bronze"
landing_volume = "landing"

base_path = f"/Volumes/{catalog_name}/default/{landing_volume}"

In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {bronze_db_name}")

## Ingestão de Arquivos CSV

Função genérica para ler arquivos CSV da Landing Zone e gravar como tabelas Delta na camada Bronze.
Cada arquivo é processado com adição de timestamp de ingestão e gravado em formato Delta gerenciado.

In [0]:
def ingest_csv(file_name: str, table_name: str):
    file_path = f"{base_path}/{file_name}"
    full_table_name = f"{catalog_name}.{bronze_db_name}.{table_name}"
    
    print(f"Ingestão: {file_name} → {full_table_name}")
    
    try:
        # Leitura do arquivo CSV com inferência de schema
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(file_path)
        
        # Adição de timestamp de ingestão para rastreabilidade
        df = df.withColumn("ingestion_timestamp", current_timestamp())
        
        # Gravação como tabela Delta gerenciada
        df.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(full_table_name)
        
        print(f"Sucesso: {full_table_name} ({df.count()} registros)")
    
    except Exception as e:
        print(f"Erro: {file_name} - {e}")

### Execução da Ingestão

Processamento de todos os arquivos CSV da Landing Zone para a camada Bronze.
Nomenclatura: `ft_` para tabelas fato e `dm_` para dimensões.

In [0]:
ingest_csv("base_atendentes.csv", "ft_atendentes")
ingest_csv("Chamados_Hora.CSV", "ft_chamados_hora")
ingest_csv("base_motivos.csv", "ft_motivos")
ingest_csv("canais.csv", "dm_canais")
ingest_csv("chamadas.csv", "ft_chamados")
ingest_csv("clientes.csv", "dm_clientes")
ingest_csv("custos.csv", "ft_custos")
ingest_csv("pesquisa_satisfacao.csv", "ft_pesquisa_satisfacao")

print("Ingestão CSV concluída")

## Validação de Tabelas Bronze

Prévia visual das tabelas criadas na camada Bronze.
Exibe as primeiras linhas de cada tabela para verificação de integridade dos dados.

In [0]:
def preview_tables(tables: list, limit_rows: int = 5):
    for table in tables:
        full_table_name = f"{catalog_name}.{bronze_db_name}.{table}"
        print(f"Tabela: {full_table_name}")
        try:
            # Carrega e exibe preview da tabela
            df = spark.table(full_table_name).limit(limit_rows)
            display(df)
        except Exception as e:
            print(f"Erro: {full_table_name} - {e}")

### Lista de Tabelas

Definição de todas as tabelas criadas na camada Bronze para validação.

In [0]:
bronze_tables = [
    "ft_atendentes",
    "ft_chamados_hora",
    "ft_motivos",
    "dm_canais",
    "ft_chamados",
    "dm_clientes",
    "ft_custos",
    "ft_pesquisa_satisfacao"
]

preview_tables(bronze_tables)

## Validação Final

Resumo da ingestão com contagem de registros por tabela.
Permite verificar rapidamente se todas as tabelas foram carregadas corretamente.

In [0]:
print("="*80)
print("RESUMO - CAMADA BRONZE")
print("="*80)

# Itera sobre todas as tabelas e exibe contagem de registros
for table in bronze_tables:
    full_name = f"{catalog_name}.{bronze_db_name}.{table}"
    try:
        count = spark.table(full_name).count()
        print(f"{table}: {count:,} registros")
    except Exception as e:
        print(f"{table}: ERRO - {e}")

print("="*80)